In [1]:
#DONE Initial Solution Finder

import DelimitedFiles
import HiGHS
import JuMP
import Random
import Statistics

using DelimitedFiles
using JuMP
using Random
using Statistics

function read_data(filename::String)
    data = DelimitedFiles.readdlm(filename)
    rows, columns = data[2:end, 1], data[1, 2:end]
    return Containers.DenseAxisArray(data[2:end, 2:end], rows, columns)
end

transport_cost_matrix = read_data(joinpath(@__DIR__, "transp.txt"))

function solve_transportation_problem(data::Containers.DenseAxisArray)
    # Get the set of supplies and demands
    O, D = axes(data)
    # Drop the EVACUEES and VACANCIES nodes from our sets
    O, D = setdiff(O, ["VACANCIES"]), setdiff(D, ["EVACUEES"])
    model = Model(HiGHS.Optimizer)
    set_silent(model)
    @variable(model, x[o in O, d in D] >= 0)
    # Remove arcs with "." cost by fixing them to 0.0.
    for o in O, d in D
        if data[o, d] == "."
            fix(x[o, d], 0.0; force = true)
        end
    end
    @objective(
        model,
        Min,
        sum(data[o, d] * x[o, d] for o in O, d in D if data[o, d] != "."),
    )
    @constraint(model, [o in O], sum(x[o, :]) == data[o, "EVACUEES"])
    @constraint(model, [d in D], sum(x[:, d]) <= data["VACANCIES", d])
    optimize!(model)
    
    
    # Construct the solution matrix
    solution_matrix = zeros(length(O), length(D))  # Initialize with zeros

    for (i, o) in enumerate(O)
        for (j, d) in enumerate(D)
            solution_matrix[i, j] = value(x[o, d])
        end
    end
    
    # Pretty print the solution in the format of the input
    print("    ", join(lpad.(D, 7, ' ')))
    for o in O
        print("\n", o)
        for d in D
            if isapprox(value(x[o, d]), 0.0; atol = 1e-6)
                print("      .")
            else
                print(" ", lpad(value(x[o, d]), 6, ' '))
            end
        end
    end
    return solution_matrix
end

print("\nSolution:\n")
solution_matrix = solve_transportation_problem(transport_cost_matrix)
println(solution_matrix)


Solution:
     広瀬中 仙台青 蒲町小 長町中 七郷小 南光台 新田小 仙台高 錦ケ丘 上杉山 宮城野 仙台商 岩切小 榴岡小 東長町 富沢小 富沢中   TSSW   TSDA   TSMP   TSTP   TSMN   TSHW   TSMS   TSH1   TSH2   TSH3
TTWR      .      .      .  894.0      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .
ZUHO      .      .      .   60.0      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .
OSAK      .      .      .      .      .      .      . 100000.0      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .
TDAI      .      .      . 5000.0      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .
AERT      .      .      .      .      .      .      .      .      . 

In [2]:
#DONE Total Cost Calculation
solution_cost = transport_cost_matrix.data[1:end-1,1:end-1]
matching_indicies = findall(x-> x == ".", solution_cost)
solution_cost[matching_indicies] .= 0
#solution_matrix .* solution_cost
total_cost_matrix = solution_cost.*solution_matrix
running_total = sum(total_cost_matrix)
initial_running_total = running_total

507759.75999999995

In [3]:
#DONE Definitions
ratios = solution_matrix ./ transport_cost_matrix.data[1:end-1, end]

#define original_population variable
original_population = transport_cost_matrix.data[1:end-1,end]

#define capacity variable
capacity = transport_cost_matrix.data[end, 1:end-1]

27-element Vector{Any}:
 152320
 149550
 129500
 163780
 121950
 132610
  21261
 104640
  14474
  14445
  14733
  24328
  16014
      ⋮
  15111
  18578
 999999
 999999
 999999
 999999
 999999
 999999
 999999
 999999
 999999
 999999

In [4]:
#DONE define perturbed population and new solution matrix
epsilon_range = 0.03
perturbation_range = 1.0 .+ epsilon_range * 2 * (0.5 .- rand(size(original_population)[1]))
println("perturbation_range:")
println(perturbation_range)
perturbed_population = original_population .* perturbation_range
new_solution_matrix_decimal = perturbed_population .* ratios
new_solution_matrix = round.(new_solution_matrix_decimal)
println("new_solution_matrix:")
println(new_solution_matrix)

perturbation_range:
[1.0000156827899924, 0.9887501028146128, 0.9847112245378554, 0.9797129586993657, 1.0114121584172449, 0.9754120638277995, 0.9985773260203321, 1.0261506410556478, 1.0228483605116996, 1.0177047470521654, 1.003361782313219, 1.0111016682919354, 1.0084472670064193, 0.9808463898486935, 0.9733495262935209, 1.0065151185565144, 0.9745617985412971, 0.9916923375107983, 1.0147479123978989, 0.973303056208468, 1.020878127825119, 0.9997976194237259, 1.0229092016226704, 1.0001958532531128]
new_solution_matrix:
[0.0 0.0 0.0 894.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 59.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 98471.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 4899.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.

In [5]:
#DONE MONTE CARLO!
shelter_matrix = read_data(joinpath(@__DIR__, "shelter_matrix.txt"))
#println(shelter_matrix)

assigned_shelters_pop = sum(new_solution_matrix, dims=1)

negativity_check = (transpose(capacity) - assigned_shelters_pop)[1:16]

println(negativity_check .< 0)
overflow_shelters = findall(<(0), negativity_check) #indices of shelters that are over capacity

function minfinder(x)
    if x > 0
        return x
    else 
        return 999999 #return huge number so 0 isn't chosen
    end
end
println(overflow_shelters)
for i in overflow_shelters
    #println("Row ",i,", Overflow Value: ",-testing123[i]) THIS WAS WHICH ONE IS OVERFLOWING!!!
    row = shelter_matrix.data[i,:]
    (minvalue,minindex) = findmin(minfinder,row)
    println(minvalue)
    println(minindex)
    println(row)
    
    overflow_population = assigned_shelters_pop[i] - capacity[i]
    assigned_shelters_pop[minindex] += overflow_population
    assigned_shelters_pop[i] -= overflow_population
    running_total += overflow_population * minvalue
     
    
    is_pop_moving = assigned_shelters_pop[minindex] > capacity[minindex]
    println(is_pop_moving)
    looping_number = 0
    
    while is_pop_moving && looping_number <=20
        println(looping_number)
        oldindex = minindex
        row = shelter_matrix.data[oldindex,:]
        (minvalue,minindex) = findmin(minfinder,row)
        overflow_population = assigned_shelters_pop[oldindex] - capacity[oldindex]
        assigned_shelters_pop[minindex] += overflow_population
        assigned_shelters_pop[oldindex] -= overflow_population
        println(assigned_shelters_pop)
        is_pop_moving = assigned_shelters_pop[minindex] > capacity[minindex]
        println(is_pop_moving)
        running_total += overflow_population * minvalue
        looping_number += 1
    end
    if is_pop_moving
        final_overflow_count = assigned_shelters_pop[minindex] - capacity[minindex]
        print("Final Overflow Population Number:") 
        println(final_overflow_count)
    end
end
println("Total Cost:")
println(running_total)
println("Cost Difference between Final Cost and initial cost: ")
println(running_total - initial_running_total)
println("Percentage")
print(initial_running_total / running_total)


Bool[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0]
[11, 14]
0.39
21
Any[13.88, 8.22, 4.43, 4.96, 5.65, 5.98, 3.36, 7.25, 16, 2.8, 0, 6.48, 9.87, 0.91, 6.25, 7.96, 6.91, 3.43, 5.38, 6.89, 0.39, 9.16, 4.88, 0.72, 5, 5.37, 5.65]
false
0.65
21
Any[13.29, 7.63, 5.04, 4.34, 6.25, 5.69, 4.16, 6.66, 15.41, 2.23, 0.91, 7.28, 9.3, 0, 5.64, 7.35, 6.29, 2.63, 5.29, 6.27, 0.65, 8.59, 4.36, 1.33, 4.39, 4.85, 5.14]
false
Total Cost:
508084.62999999995
Cost Difference between Final Cost and initial cost: 
324.86999999999534
Percentage
0.9993605986467254

In [18]:
#randomly population pertubation for variance testing.

import JuMP
using JuMP
import Random
using Random
import DelimitedFiles
using DelimitedFiles
import Statistics
using Statistics
import HiGHS

open(joinpath(@__DIR__, "transp.txt"), "w") do io
    print(
        io,
        """
             . 広瀬中	仙台青	蒲町小	長町中	七郷小	南光台	新田小	仙台高	錦ケ丘	上杉山	宮城野	仙台商	岩切小	榴岡小	東長町	富沢小	富沢中	TSSW	TSDA	TSMP	TSTP	TSMN	TSHW	TSMS	TSH1	TSH2	TSH3  EVACUEES
          TTWR .	.	.	2.85	.	.	.	4.80	.	1.65	2.31	.	.	1.47	4.33	.	4.38	1.39	4.37	3.94	1.99	.	3.88	2.46	3.76	4.34	4.49	894
          ZUHO .	.	.	2.30	.	.	.	4.58	.	2.36	3.37	.	.	2.54	4.12	4.54	3.57	1.20	.	3.00	3.03	.	4.03	3.44	3.85	4.53	4.64	60
          OSAK .	2.05	.	.	.	.	.	1.46	.	2.65	.	.	.	4.45	.	.	.	2.06	4.17	.	.	.	.	.	.	.	.	100000
          TDAI .	.	.	2.54	.	.	.	4.82	.	1.93	2.61	.	.	1.78	4.11	4.98	4.03	1.35	4.69	3.57	2.27	.	3.77	2.70	3.63	4.25	4.39	5000
          AERT .	.	.	3.56	.	4.54	4.64	4.84	.	1.20	1.77	.	.	0.93	4.86	.	.	1.75	3.70	4.72	1.50	.	4.22	2.06	4.13	4.64	4.81	584
          TEPC .	4.81	.	3.86	.	4.33	4.78	4.47	.	0.78	2.04	.	.	1.25	.	.	.	1.58	3.36	4.92	1.82	.	4.64	2.40	4.54	.	.	12500
          SIPO .	.	.	.	.	.	.	.	.	.	.	3.68	.	.	.	.	.	.	.	.	.	4.29	.	.	.	.	.	1653
          MEDI .	3.88	.	3.89	.	4.99	.	3.45	.	0.99	3.21	.	.	2.39	.	.	.	0.57	3.65	4.54	2.97	.	.	3.53	.	.	.	346
          SSTA .	.	4.83	3.35	.	4.71	4.60	.	.	1.47	1.66	.	.	0.81	4.61	.	4.96	1.87	3.94	4.58	1.36	.	3.95	1.90	3.86	4.37	4.54	75608
          AQUA .	.	.	.	.	.	4.97	.	.	.	.	.	4.19	.	.	.	.	.	.	.	.	.	.	.	.	.	.	913
          MOPS .	.	.	.	.	.	.	.	.	.	.	.	5.09	.	.	.	.	.	.	.	.	.	.	.	.	.	.	10000
          YURT .	.	.	.	.	2.70	.	.	.	.	.	1.03	.	.	.	.	.	.	2.58	.	.	0.49	.	.	.	.	.	11467
          CAST .	4.51	.	2.90	.	.	.	3.87	.	2.68	4.27	.	.	3.42	4.85	4.72	3.77	1.15	.	3.01	3.95	.	4.95	4.41	4.76	.	.	87
          RAKU .	.	2.96	3.98	4.13	4.78	3.19	.	.	3.13	0.68	.	.	1.21	4.39	.	.	3.79	4.74	.	0.67	.	3.24	0.13	3.27	3.48	3.68	27000
          TLSW 9.36	4.01	6.62	3.51	7.86	5.56	6.33	3.49	9.50	1.55	3.51	7.02	8.45	2.66	5.30	5.65	4.69	0.00	4.21	4.02	3.24	6.61	5.10	3.77	4.95	5.59	5.72   999999
          TLDA .	4.38	7.45	7.22	8.38	1.86	4.90	4.57	.	2.80	4.08	3.21	5.62	3.91	8.53	9.66	8.71	4.23	0.00	8.16	4.17	2.66	7.74	4.63	7.69	8.09	8.28   999999
          TLMP .	7.36	7.04	1.70	8.21	9.21	8.68	6.67	9.68	5.35	5.76	.	.	5.11	3.11	1.75	0.89	4.02	8.13	0.00	5.41	.	4.03	5.60	3.80	4.45	4.45   999999
          TLTP .	6.56	3.63	3.96	4.79	4.38	3.37	6.26	.	2.47	0.36	7.35	5.98	0.58	4.66	6.50	5.66	3.23	4.16	5.47	0.00	6.78	3.64	0.58	3.64	3.95	4.14   999999
          TLMN .	5.20	9.87	9.82	.	3.07	6.79	5.70	.	5.35	6.67	0.62	6.40	6.57	.	.	.	6.62	2.66	.	6.80	0.00	.	7.22	.	.	.   999999
          TLHW .	9.02	3.23	2.47	4.29	8.01	5.98	8.50	.	5.38	3.86	.	8.86	3.81	1.38	4.04	3.62	5.05	7.72	4.03	3.64	.	0.00	3.37	0.23	0.50	0.62   999999
          TLMS .	7.13	3.05	4.07	4.20	4.65	3.12	6.83	.	3.05	0.55	7.77	5.86	1.14	4.51	6.56	5.77	3.76	4.61	5.66	0.58	7.19	3.37	0.00	3.40	3.62	3.81   999999
          TLH1 .	8.88	3.43	2.24	4.50	8.02	6.09	8.35	.	5.29	3.87	.	8.96	3.78	1.23	3.84	3.40	4.90	7.68	3.80	3.64	.	0.23	3.40	0.00	0.71	0.79   999999
          TLH2 .	9.49	3.02	2.94	3.99	8.27	6.04	8.98	.	5.80	4.13	.	8.92	4.17	1.59	4.32	3.98	5.53	8.07	4.45	3.95	.	0.50	3.62	0.71	0.00	0.20   999999
          TLH3 .	9.63	3.16	2.97	4.09	8.46	6.23	9.11	.	5.97	4.33	.	9.10	4.35	1.50	4.23	3.94	5.66	8.25	4.44	4.14	.	0.62	3.81	0.79	0.20	0.00   999999
        VACANCIES 152320 149550 129500 163780 121950 132610 21261 104640 14474 14445 14733 24328 16014 13779 12595 15111 18578 999999 999999 999999 999999 999999 999999 999999 999999 999999 999999 0
        """,
    )
    return
end

function read_data(filename::String)
    data = DelimitedFiles.readdlm(filename)
    rows, columns = data[2:end, 1], data[1, 2:end]
    return Containers.DenseAxisArray(data[2:end, 2:end], rows, columns)
end

data = read_data(joinpath(@__DIR__, "transp.txt"))
print(data)

function solve_transportation_problem(data::Containers.DenseAxisArray)
    # Get the set of supplies and demands
    O, D = axes(data)
    # Drop the EVACUEES and VACANCIES nodes from our sets
    O, D = setdiff(O, ["VACANCIES"]), setdiff(D, ["EVACUEES"])
    model = Model(HiGHS.Optimizer)
    set_silent(model)
    @variable(model, x[o in O, d in D] >= 0)
    # Remove arcs with "." cost by fixing them to 0.0.
    for o in O, d in D
        if data[o, d] == "."
            fix(x[o, d], 0.0; force = true)
        end
    end
    @objective(
        model,
        Min,
        sum(data[o, d] * x[o, d] for o in O, d in D if data[o, d] != "."),
    )
    @constraint(model, [o in O], sum(x[o, :]) == data[o, "EVACUEES"])
    @constraint(model, [d in D], sum(x[:, d]) <= data["VACANCIES", d])
    optimize!(model)
    # Pretty print the solution in the format of the input
    print("    ", join(lpad.(D, 7, ' ')))
    for o in O
        print("\n", o)
        for d in D
            if isapprox(value(x[o, d]), 0.0; atol = 1e-6)
                print("      .")
            else
                print(" ", lpad(value(x[o, d]), 6, ' '))
            end
        end
    end
    return
end
#-------------
# Specify the number of Monte Carlo trials and epsilon range
num_trials = 50  # Adjust this as needed
epsilon_range = 0.07  # range for population perturbations

# Define a function to perturb the population data with randomization
function perturb_population(data::Containers.DenseAxisArray, perturbations::Dict{String, Float64})
    perturbed_data = deepcopy(data)
print("printing perturbations")
    for (o, perturbation) in perturbations
        # Apply the perturbation to the population
        original_population = data[o, "EVACUEES"]
        perturbed_population = original_population * (1.0 + perturbation)
        println(perturbation)

        # Update the perturbed population in the data
        perturbed_data[o, "EVACUEES"] = perturbed_population
    end

    return perturbed_data
end

# Define a function to run the Monte Carlo simulation with population perturbations
function run_monte_carlo_simulation(data::Containers.DenseAxisArray, num_trials::Int, epsilon_range::Float64)
    # Create an array to store the results from each trial
    total_distances = Float64[]

    # Loop through the specified number of trials
    for trial in 1:num_trials
        println("\nTrial $trial:")

        # Set a new random seed for each trial to ensure different perturbations
        Random.seed!(trial)

        # Create a random perturbation for each DL
        perturbations = Dict{String, Float64}()
        total_perturbation = 0.0  # Initialize total perturbation for this trial
        for o in axes(data)[1]
            if o != "EVACUEES" && o != "VACANCIES"  # Exclude "VACANCIES" from perturbation
                # Generate a random perturbation within the epsilon range
                perturbation = (2.0 * epsilon_range * rand() - epsilon_range)  # Random float within the range
                perturbations[o] = perturbation
                println("Perturbation for $o: $perturbation")

                # Calculate perturbation contribution to total perturbation
                original_population = data[o, "EVACUEES"]
                perturbation_contribution = original_population * perturbation
                total_perturbation += perturbation_contribution
            end
        end

        # Print the total perturbation for this trial
        println("Total Perturbation for Trial $trial: $total_perturbation")

        # Perturb the population data using the perturbations
        perturbed_data = perturb_population(data, perturbations)

        # Solve the transportation problem for the perturbed data
        solve_transportation_problem(perturbed_data)

        # Calculate and record the total distance for this trial
        total_distance = calculate_total_distance(data, perturbed_data)
        push!(total_distances, total_distance)
    end
    return total_distances
end

# Run the Monte Carlo simulation
results = run_monte_carlo_simulation(data, num_trials, epsilon_range)

# Output the results
println("\nSimulation Results:")
println("Total Distances for Each Trial: ", results)

average_total_cost = mean(results)
println("Average Total Cost for All Trials: $average_total_cost")

2-dimensional DenseAxisArray{Any,2,...} with index sets:
    Dimension 1, Any["TTWR", "ZUHO", "OSAK", "TDAI", "AERT", "TEPC", "SIPO", "MEDI", "SSTA", "AQUA"  …  "TLDA", "TLMP", "TLTP", "TLMN", "TLHW", "TLMS", "TLH1", "TLH2", "TLH3", "VACANCIES"]
    Dimension 2, Any["広瀬中", "仙台青", "蒲町小", "長町中", "七郷小", "南光台", "新田小", "仙台高", "錦ケ丘", "上杉山"  …  "TSDA", "TSMP", "TSTP", "TSMN", "TSHW", "TSMS", "TSH1", "TSH2", "TSH3", "EVACUEES"]
And data, a 25×28 Matrix{Any}:
       "."        "."        "."       2.85        "."        "."       "."       4.8        "."      1.65      2.31       "."       "."      1.47      4.33       "."      4.38       1.39       4.37       3.94       1.99        "."       3.88       2.46       3.76       4.34       4.49     894
       "."        "."        "."       2.3         "."        "."       "."       4.58       "."      2.36      3.37       "."       "."      2.54      4.12      4.54      3.57       1.2         "."       3.0        3.03        "."       4.03       3

In [ ]:
#Delete this cell???
#always run cell above first
import Random
using Random
max_deviation = 0.05
Random.seed!(123)
rand()

O, D = axes(transport_cost_matrix)
# Drop the EVACUEES and VACANCIES nodes from our sets
O, D = setdiff(O, ["VACANCIES"]), setdiff(D, ["EVACUEES"])
# Remove arcs with "." cost by fixing them to 0.0.
for o in O, d in D
    if data[o, d] == "."
        data[o,d] = 0.0
    end
    new_pop = 1-rand()*max_deviation
    print(new_pop)
    data[o,d] *= new_pop
end
print(rand())
print(data)
    
#Shrink (*95-100%), OR, make sure shelter cap >> DL pop
#wrap the entire cell in a for loop. Keep track of all costs. average out the cost. 

0.97065966212733260.9554560650953610.99045466504871180.97371688042289760.98047058622843270.99775909974912550.95333233563614170.97097200090627290.98363810603718580.97365020406015070.95818857124739240.99795469319861540.97673992473093880.98186753367907790.9948897696755620.96399487202548350.9713190378765680.96677657606365350.985231674762260.98617012769125170.9508282144440080.95595512545920970.98829915971129720.98095253103569460.9934028133730250.99558544904338610.98432475427461350.97681951278375420.96431820387568960.98970375452566450.9954724417891110.97090438288061780.98442762496474730.99394262397409360.9897735091339820.98066549185455230.99907140002050310.99639096381492960.95428767070281030.98063129688732180.96140418220776110.98113083702836230.95622225002201910.98677159298190340.95490613377347920.95676534539844150.96338809243910150.99484015767495850.97062075318955740.95952836220292690.98880795986397360.9570737769507660.98122915381477430.96616464704802270.95395577879366440.96467694292159540.

0.9608077415820610.96915134403676870.97254922142306430.96095355090467370.9781978252677380.99499535341738870.99350191683422930.99329511331079780.9985269892167250.98552034388366650.98667284518223930.99061381213589870.9683486242512580.95758265298192070.96788090163961480.9506152747727570.99552876719164160.95766449011250930.95733813776397790.9516805675900780.99393766130618380.96789025030619840.98986278391549460.95374788749755680.95050747692205560.9845349451778150.95394274675347420.96830219273232120.98780986808120260.9716713187787040.95471367809694640.98229739761684230.97914631108536070.96000801913395670.96875429339304460.96683739006483350.98413570321158380.99823001583001480.99108368699480150.9528037168677470.98464833530753550.95265434480959820.98274578817101490.98056310563148890.9600247460477970.95788400072404560.97723000806789940.96622520213577370.97868340391865580.96159587292469060.99442235186634680.96187266166298080.98339847650024360.99069575224952190.96928736490166990.95333940557414950.

      0.0                     4.3109934484134795       0.0                      2.8582219085567777       0.0                      0.0                     0.0                      3.8024708367664393      0.0                    2.6547766828614727      4.157040117840961        0.0                     0.0                    3.3035111349688586      4.67242801409108        4.551204797287443       3.7329043999817624       1.1264573478963644       0.0                      2.9460554863561677       3.7844104864510175       0.0                      4.753091080710651         4.19810311591398          4.752744630106487         0.0                       0.0                     87
      0.0                     0.0                      2.892537247518189        3.9364563649159656       4.113741243761644        4.741025002753437       3.1641620689280643       0.0                     0.0                    3.0389691532785106      0.6512067755101444       0.0                     0.0                    1.1

In [ ]:
#DONE VISUALIZATION
using Conda
Conda.runconda(`install folium -c conda-forge`)

import PyCall
using PyCall
fol = pyimport("folium")

min_lon, max_lon = 140.4, 141.14
min_lat, max_lat = 38.1, 38.5

# Create a folium Map object
m = fol.Map(
    max_bounds=true,
    location=[38.150223, 140.869415],
    zoom_start=10,
    min_lat=min_lat,
    max_lat=max_lat,
    min_lon=min_lon,
    max_lon=max_lon,
)

# Add CircleMarkers to the map
fol.CircleMarker([max_lat, min_lon], tooltip="Upper Left Corner").add_to(m)
fol.CircleMarker([min_lat, min_lon], tooltip="Lower Left Corner").add_to(m)
fol.CircleMarker([min_lat, max_lon], tooltip="Lower Right Corner").add_to(m)
fol.CircleMarker([max_lat, max_lon], tooltip="Upper Right Corner").add_to(m)

#Icons
fg = fol.FeatureGroup(name="Icon collection").add_to(m)
fol.Marker(location=(38.25523, 140.869415)).add_to(fg)

# Load and display the GeoJSON file
sendai_border = "/Users/jakechon/Downloads/04100_sendai-shi_2022_other_1_op/sendai_border.geojson"
fol.GeoJson(sendai_border).add_to(m)

fol.LayerControl().add_to(m)

#Population Distribution use https://www.researchgate.net/publication/329350011_Dynamic_Integrated_Model_for_Disaster_Management_and_Socioeconomic_Analysis_DIM2SEA

#UX
fol.GeoJson(sendai_border, zoom_on_click=true).add_to(m)#make it so that it isn't only the lines

# Display the map
m


In [18]:
#Derivative Calculation (Pathway Cost)

function calculate_numerical_derivative(data::JuMP.Containers.DenseAxisArray{Any, 2}, epsilon::Float64 = 1)
    O, D = axes(data)
    O, D = setdiff(O, ["VACANCIES"]), setdiff(D, ["EVACUEES"])
    num_rows, num_cols = length(O), length(D)
    derivative_matrix = zeros(Float64, num_rows, num_cols)

    for (i, o) in enumerate(O), (j, d) in enumerate(D)
        original_cost = data[o, d]

        if original_cost isa Number
            # Increase the cost slightly and re-solve
            data[o, d] = original_cost + epsilon
            increased_cost_model = Model(HiGHS.Optimizer)
            set_silent(increased_cost_model)
            @variable(increased_cost_model, x[ii in O, jj in D] >= 0)

            for ii in O, jj in D
                if data[ii, jj] == "."
                    fix(x[ii, jj], 0.0; force = true)
                end
            end

            @objective(
                increased_cost_model,
                Min,
                sum(data[ii, jj] * x[ii, jj] for ii in O, jj in D if data[ii, jj] isa Number),
            )

            @constraint(increased_cost_model, [ii in O], sum(x[ii, :]) == data[ii, "EVACUEES"])
            @constraint(increased_cost_model, [jj in D], sum(x[:, jj]) <= data["VACANCIES", jj])

            optimize!(increased_cost_model)
            increased_cost_objective = objective_value(increased_cost_model)

            # Decrease the cost slightly and re-solve
            data[o, d] = original_cost - epsilon
            decreased_cost_model = Model(HiGHS.Optimizer)
            set_silent(decreased_cost_model)
            @variable(decreased_cost_model, x[ii in O, jj in D] >= 0)

            for ii in O, jj in D
                if data[ii, jj] == "."
                    fix(x[ii, jj], 0.0; force = true)
                end
            end

            @objective(
                decreased_cost_model,
                Min,
                sum(data[ii, jj] * x[ii, jj] for ii in O, jj in D if data[ii, jj] isa Number),
            )

            @constraint(decreased_cost_model, [ii in O], sum(x[ii, :]) == data[ii, "EVACUEES"])
            @constraint(decreased_cost_model, [jj in D], sum(x[:, jj]) <= data["VACANCIES", jj])

            optimize!(decreased_cost_model)
            decreased_cost_objective = objective_value(decreased_cost_model)

            # Calculate the numerical derivative
            derivative = (increased_cost_objective - decreased_cost_objective) / (2 * epsilon)
            derivative_matrix[i, j] = derivative

            # Reset the original cost
            data[o, d] = original_cost
        end
    end

    return derivative_matrix
end

# Calculate and print the numerical derivative matrix
epsilon = 1.0
derivative_matrix = calculate_numerical_derivative(transport_cost_matrix, epsilon)
println("\nNumerical Derivative Matrix for Pathway:")
for row in eachrow(derivative_matrix)
    println(row)
end

# Derivative Calculation (Shelter Capacity)

function calculate_numerical_capacity_derivative(data::JuMP.Containers.DenseAxisArray{Any, 2}, epsilon::Float64 = 1000)
    O, D = axes(data)
    O, D = setdiff(O, ["VACANCIES"]), setdiff(D, ["EVACUEES"])
    num_rows, num_cols = length(O), length(D)
    derivative_matrix = zeros(Float64, num_rows, num_cols)

    for (i, o) in enumerate(O), (j, d) in enumerate(D)
        original_capacity = data["VACANCIES", d]

        if original_capacity isa Number
            # Increase the capacity slightly and re-solve
            data["VACANCIES", d] = original_capacity + epsilon
            increased_capacity_model = Model(HiGHS.Optimizer)
            set_silent(increased_capacity_model)
            @variable(increased_capacity_model, x[ii in O, jj in D] >= 0)

            for ii in O, jj in D
                if data[ii, jj] == "."
                    fix(x[ii, jj], 0.0; force = true)
                end
            end

            @objective(
                increased_capacity_model,
                Min,
                sum(data[ii, jj] * x[ii, jj] for ii in O, jj in D if data[ii, jj] isa Number),
            )

            @constraint(increased_capacity_model, [ii in O], sum(x[ii, :]) == data[ii, "EVACUEES"])
            @constraint(increased_capacity_model, [jj in D], sum(x[:, jj]) <= data["VACANCIES", jj])

            optimize!(increased_capacity_model)
            increased_capacity_objective = objective_value(increased_capacity_model)

            # Decrease the capacity slightly and re-solve
            data["VACANCIES", d] = original_capacity - epsilon
            decreased_capacity_model = Model(HiGHS.Optimizer)
            set_silent(decreased_capacity_model)
            @variable(decreased_capacity_model, x[ii in O, jj in D] >= 0)

            for ii in O, jj in D
                if data[ii, jj] == "."
                    fix(x[ii, jj], 0.0; force = true)
                end
            end

            @objective(
                decreased_capacity_model,
                Min,
                sum(data[ii, jj] * x[ii, jj] for ii in O, jj in D if data[ii, jj] isa Number),
            )

            @constraint(decreased_capacity_model, [ii in O], sum(x[ii, :]) == data[ii, "EVACUEES"])
            @constraint(decreased_capacity_model, [jj in D], sum(x[:, jj]) <= data["VACANCIES", jj])

            optimize!(decreased_capacity_model)
            decreased_capacity_objective = objective_value(decreased_capacity_model)

            # Calculate the numerical derivative
            derivative = (increased_capacity_objective - decreased_capacity_objective) / (2 * epsilon)
            derivative_matrix[i, j] = derivative

            # Reset the original capacity
            data["VACANCIES", d] = original_capacity
        end
    end

    return derivative_matrix
end

# Calculate and print the numerical derivative matrix for shelter capacity changes
epsilon = 1000.0
capacity_derivative_matrix = calculate_numerical_capacity_derivative(transport_cost_matrix, epsilon)
println("\nNumerical Derivative Matrix for Shelter Capacity Changes:")
for row in eachrow(capacity_derivative_matrix)
    println(row)
end


Numerical Derivative Matrix for Pathway:
[0.0, 0.0, 0.0, 750.960000000021, 0.0, 0.0, 0.0, 0.0, 0.0, 143.03999999997905, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 60.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 20500.0, 0.0, 0.0, 0.0, 0.0, 0.0, 79500.00000000003, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 5000.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 151.8399999999674, 0.0, 0.0, 0.0, 0.0, 0.0, 405.88000000003376, 8.759999999951106, 0.0, 0.0, 178.11999999996624, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12500.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0,